# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

# Eager mode on

In [2]:
tf.enable_eager_execution()

# Checkpoint function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

# Create model (class version)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

# Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.4975, train_loss: 0.33036286, train_accuracy: 0.3359, test_Accuracy: 0.2291
Epoch: [ 0] [    1/  468] time: 0.7914, train_loss: 0.29299241, train_accuracy: 0.3672, test_Accuracy: 0.3296
Epoch: [ 0] [    2/  468] time: 1.0807, train_loss: 0.29535800, train_accuracy: 0.4297, test_Accuracy: 0.3945
Epoch: [ 0] [    3/  468] time: 1.3561, train_loss: 0.30906296, train_accuracy: 0.5078, test_Accuracy: 0.4767
Epoch: [ 0] [    4/  468] time: 1.6425, train_loss: 0.26806700, train_accuracy: 0.6250, test_Accuracy: 0.5505
Epoch: [ 0] [    5/  468] time: 1.9350, train_loss: 0.32235762, train_accuracy: 0.5781, test_Accuracy: 0.5983
Epoch: [ 0] [    6/  468] time: 2.2008, train_loss: 0.26554614, train_accuracy: 0.6641, test_Accuracy: 0.6494
Epoch: [ 0] [    7/  468] time: 2.4638, train_loss: 0.20839277, train_accuracy: 0.7500, test_Accuracy: 0.6980
Epoch: [ 0] [    8/  468] time: 2.7304,

Epoch: [ 0] [   74/  468] time: 21.6645, train_loss: 0.17417696, train_accuracy: 0.9297, test_Accuracy: 0.9382
Epoch: [ 0] [   75/  468] time: 21.9596, train_loss: 0.19491014, train_accuracy: 0.9531, test_Accuracy: 0.9366
Epoch: [ 0] [   76/  468] time: 22.2544, train_loss: 0.26912385, train_accuracy: 0.9141, test_Accuracy: 0.9390
Epoch: [ 0] [   77/  468] time: 22.5362, train_loss: 0.10218343, train_accuracy: 0.9609, test_Accuracy: 0.9426
Epoch: [ 0] [   78/  468] time: 22.8188, train_loss: 0.14223242, train_accuracy: 0.9531, test_Accuracy: 0.9461
Epoch: [ 0] [   79/  468] time: 23.0969, train_loss: 0.10368903, train_accuracy: 0.9609, test_Accuracy: 0.9480
Epoch: [ 0] [   80/  468] time: 23.3720, train_loss: 0.11104535, train_accuracy: 0.9453, test_Accuracy: 0.9463
Epoch: [ 0] [   81/  468] time: 23.6597, train_loss: 0.13129853, train_accuracy: 0.9531, test_Accuracy: 0.9456
Epoch: [ 0] [   82/  468] time: 23.9430, train_loss: 0.14899245, train_accuracy: 0.9453, test_Accuracy: 0.9447
E

Epoch: [ 0] [  148/  468] time: 42.5741, train_loss: 0.21957263, train_accuracy: 0.9062, test_Accuracy: 0.9512
Epoch: [ 0] [  149/  468] time: 42.8479, train_loss: 0.07872219, train_accuracy: 0.9844, test_Accuracy: 0.9515
Epoch: [ 0] [  150/  468] time: 43.1197, train_loss: 0.06321964, train_accuracy: 0.9531, test_Accuracy: 0.9535
Epoch: [ 0] [  151/  468] time: 43.3962, train_loss: 0.07186382, train_accuracy: 0.9766, test_Accuracy: 0.9552
Epoch: [ 0] [  152/  468] time: 43.6747, train_loss: 0.03968059, train_accuracy: 0.9922, test_Accuracy: 0.9566
Epoch: [ 0] [  153/  468] time: 43.9511, train_loss: 0.10695499, train_accuracy: 0.9688, test_Accuracy: 0.9570
Epoch: [ 0] [  154/  468] time: 44.2276, train_loss: 0.09737165, train_accuracy: 0.9688, test_Accuracy: 0.9585
Epoch: [ 0] [  155/  468] time: 44.5021, train_loss: 0.05561279, train_accuracy: 0.9844, test_Accuracy: 0.9588
Epoch: [ 0] [  156/  468] time: 44.7784, train_loss: 0.08986782, train_accuracy: 0.9609, test_Accuracy: 0.9595
E

Epoch: [ 0] [  222/  468] time: 63.2361, train_loss: 0.10099342, train_accuracy: 0.9219, test_Accuracy: 0.9602
Epoch: [ 0] [  223/  468] time: 63.5120, train_loss: 0.08429043, train_accuracy: 0.9375, test_Accuracy: 0.9602
Epoch: [ 0] [  224/  468] time: 63.7831, train_loss: 0.04090427, train_accuracy: 0.9922, test_Accuracy: 0.9602
Epoch: [ 0] [  225/  468] time: 64.0724, train_loss: 0.06599039, train_accuracy: 0.9844, test_Accuracy: 0.9592
Epoch: [ 0] [  226/  468] time: 64.3587, train_loss: 0.06344657, train_accuracy: 0.9766, test_Accuracy: 0.9578
Epoch: [ 0] [  227/  468] time: 64.6332, train_loss: 0.13878784, train_accuracy: 0.9453, test_Accuracy: 0.9571
Epoch: [ 0] [  228/  468] time: 64.9136, train_loss: 0.16873755, train_accuracy: 0.9453, test_Accuracy: 0.9558
Epoch: [ 0] [  229/  468] time: 65.1940, train_loss: 0.11658402, train_accuracy: 0.9609, test_Accuracy: 0.9554
Epoch: [ 0] [  230/  468] time: 65.4774, train_loss: 0.08872084, train_accuracy: 0.9688, test_Accuracy: 0.9551
E

Epoch: [ 0] [  296/  468] time: 83.8589, train_loss: 0.05977339, train_accuracy: 0.9844, test_Accuracy: 0.9626
Epoch: [ 0] [  297/  468] time: 84.1454, train_loss: 0.07566610, train_accuracy: 0.9688, test_Accuracy: 0.9615
Epoch: [ 0] [  298/  468] time: 84.4224, train_loss: 0.12614591, train_accuracy: 0.9688, test_Accuracy: 0.9578
Epoch: [ 0] [  299/  468] time: 84.7060, train_loss: 0.14288053, train_accuracy: 0.9688, test_Accuracy: 0.9551
Epoch: [ 0] [  300/  468] time: 84.9845, train_loss: 0.05065518, train_accuracy: 0.9766, test_Accuracy: 0.9531
Epoch: [ 0] [  301/  468] time: 85.2610, train_loss: 0.08209319, train_accuracy: 0.9609, test_Accuracy: 0.9519
Epoch: [ 0] [  302/  468] time: 85.5373, train_loss: 0.10234933, train_accuracy: 0.9531, test_Accuracy: 0.9523
Epoch: [ 0] [  303/  468] time: 85.8163, train_loss: 0.12575126, train_accuracy: 0.9609, test_Accuracy: 0.9522
Epoch: [ 0] [  304/  468] time: 86.0927, train_loss: 0.14946353, train_accuracy: 0.9609, test_Accuracy: 0.9549
E

Epoch: [ 0] [  370/  468] time: 104.2466, train_loss: 0.07196055, train_accuracy: 0.9688, test_Accuracy: 0.9675
Epoch: [ 0] [  371/  468] time: 104.5191, train_loss: 0.07451864, train_accuracy: 0.9688, test_Accuracy: 0.9679
Epoch: [ 0] [  372/  468] time: 104.7896, train_loss: 0.09573793, train_accuracy: 0.9766, test_Accuracy: 0.9670
Epoch: [ 0] [  373/  468] time: 105.0653, train_loss: 0.07326534, train_accuracy: 0.9609, test_Accuracy: 0.9675
Epoch: [ 0] [  374/  468] time: 105.3373, train_loss: 0.12505338, train_accuracy: 0.9766, test_Accuracy: 0.9670
Epoch: [ 0] [  375/  468] time: 105.6107, train_loss: 0.13861583, train_accuracy: 0.9609, test_Accuracy: 0.9648
Epoch: [ 0] [  376/  468] time: 105.8843, train_loss: 0.05679125, train_accuracy: 0.9844, test_Accuracy: 0.9636
Epoch: [ 0] [  377/  468] time: 106.1599, train_loss: 0.04744723, train_accuracy: 0.9766, test_Accuracy: 0.9639
Epoch: [ 0] [  378/  468] time: 106.4317, train_loss: 0.05268546, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 124.8562, train_loss: 0.04957449, train_accuracy: 0.9688, test_Accuracy: 0.9721
Epoch: [ 0] [  445/  468] time: 125.1316, train_loss: 0.14170647, train_accuracy: 0.9531, test_Accuracy: 0.9723
Epoch: [ 0] [  446/  468] time: 125.4117, train_loss: 0.04400287, train_accuracy: 0.9766, test_Accuracy: 0.9717
Epoch: [ 0] [  447/  468] time: 125.6919, train_loss: 0.05544932, train_accuracy: 0.9844, test_Accuracy: 0.9718
Epoch: [ 0] [  448/  468] time: 125.9700, train_loss: 0.08386404, train_accuracy: 0.9609, test_Accuracy: 0.9715
Epoch: [ 0] [  449/  468] time: 126.2484, train_loss: 0.09472851, train_accuracy: 0.9609, test_Accuracy: 0.9709
Epoch: [ 0] [  450/  468] time: 126.5273, train_loss: 0.06384026, train_accuracy: 0.9922, test_Accuracy: 0.9706
Epoch: [ 0] [  451/  468] time: 126.8147, train_loss: 0.03368580, train_accuracy: 0.9844, test_Accuracy: 0.9703
Epoch: [ 0] [  452/  468] time: 127.0962, train_loss: 0.10920897, train_accuracy: 0.9766, test_Accuracy:

# Test accuracy : 97.51 %